In [ ]:
import numpy
import scipy.special
import scipy.ndimage
import matplotlib.pyplot
%matplotlib inline

class neuralNetwork:

    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):

        # задати кількість вузлів у вхідному, прихованому і вихідному шарі

        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        # Матриці вагових коефіцієнтів зв'язків wih і who.
        # Вагові коефіцієнти зв'язків між вузлом i та вузлом j наступного шару позначені як w__i__j:

        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5),  (self.onodes, self.hnodes))
        # Використання сигмоїди як функції активації
        self.activation_function = lambda x: scipy.special.expit(x)
        self.inverse_activation_function = lambda x: scipy.special.logit(x)
        
        self.lr = learningrate
        pass

    def train (self, inputs_list, targets_list) :
        
        # перетворення вхідних списків у двовимірні масиви
        
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        # розрахунок вхідних сигналів для прихованого шару
        hidden_inputs = numpy.dot(self.wih, inputs)
        # розрахунок вихідних сигналів для прихованого шару
        hidden_outputs = self.activation_function(hidden_inputs)
        # розрахунок вхідних сигналів для вихідного шару
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # розрахунок вихідних сигналів для вихідного шару
        final_outputs = self.activation_function(final_inputs)
        # помилка = (цільове значення - фактичне)
        output_errors = targets - final_outputs
        # ошибки скрытого слоя - это ошибки output_errors,
        # распределенные пропорционально весовым коэффициентам связей
        # рекомбінація помилок на прихований шар
        hidden_errors = numpy.dot(self.who.T, output_errors)
        # оновлення вагових коефіціентів між прихованим і вихідним шаром через градієнтний спуск
        
        self.who += self.lr * numpy .dot ((output_errors * final_outputs * (1.0 - final_outputs)), 
                                          numpy.transpose (hidden_outputs))
        
        # оновлення вагових коефіціентів між вхідним і прихованим шаром через градієнтний спуск
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        pass



    def query(self, inputs_list):
        # Перетворити список вхідних значень в двовимірний масив
        inputs = numpy.array(inputs_list, ndmin=2).T
        # Розрахувати вхідні сигнали для прихованого шару
        hidden_inputs = numpy.dot(self.wih, inputs)
        # Розрахувати вихідні сигнали для прихованого шару
        hidden_outputs = self.activation_function(hidden_inputs)
        # Розрахувати вхідні сигнали для вихідного шару
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # Розрахувати вихідні сигнали для вихідного шару
        final_outputs = self.activation_function(final_inputs)
        return final_outputs
    
     # томографія нейромережі через обернену активаційну функцію і зворотнє поширення
    def backquery(self, targets_list):
        final_outputs = numpy.array(targets_list, ndmin=2).T
        
        final_inputs = self.inverse_activation_function(final_outputs)

        hidden_outputs = numpy.dot(self.who.T, final_inputs)
        hidden_outputs -= numpy.min(hidden_outputs)
        hidden_outputs /= numpy.max(hidden_outputs)
        hidden_outputs *= 0.98
        hidden_outputs += 0.01
        
        hidden_inputs = self.inverse_activation_function(hidden_outputs)
        
        inputs = numpy.dot(self.wih.T, hidden_inputs)
        inputs -= numpy.min(inputs)
        inputs /= numpy.max(inputs)
        inputs *= 0.98
        inputs += 0.01
        
        return inputs


input_nodes = 784
hidden_nodes = 500
output_nodes = 10
learning_rate= 0.01

n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)

data_file = open("mnist_dataset/mnist_train.csv", 'r')
training_data_list = data_file.readlines()
data_file.close()

# 10 епох навчання одними й тими самими тренувальними даними 

epochs = 10
for e in range(epochs):
    for record in training_data_list:
        
        # формування списку через розділювач
        all_values = record.split(',')
        # масшьабування даних після мітки
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        
        targets = numpy.zeros(output_nodes) + 0.01
        # створення цільового списку з маркером 0.99 у потрібному місці вектора

        targets[int(all_values[0])] =0.99
        n.train(inputs, targets)
        # поворот матриці зображення 28*28 на 10 градусів
        inputs_plusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 10, cval=0.01, order=1, reshape=False)
        n.train(inputs_plusx_img.reshape(784), targets)
        # поворот матриці зображення 28*28 на -10 градусів
        inputs_minusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), -10, cval=0.01, order=1, reshape=False)
        n.train(inputs_minusx_img.reshape(784), targets)
        
        
        pass
    pass
  


test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.closed

scorecard = []


for record in test_data_list:

    all_values = record.split(',')

    correct_label = int(all_values[0])
    

    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

    outputs = n.query(inputs)
    # отримання індексу найбільшого значення на виході, що співпадає з числом
    label = numpy.argmax(outputs)
    
    
    # формування статистики


    if (label == correct_label) :
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    pass

scorecard_array = numpy.asarray(scorecard)
print ("efficiency = ", (scorecard_array.sum() / scorecard_array.size*100))


labels = 10
for label in range(labels):
    # друк серії томографічних знимків
    targets = numpy.zeros(output_nodes) + 0.01
    targets[label] = 0.99
    print(targets)
    image_data = n.backquery(targets)
    matplotlib.pyplot.imshow(image_data.reshape(28,28), cmap='Greys', interpolation='None')


